In [1]:
import codecs
import json
import os
import random


import spacy
from spacy.tokens import DocBin, Doc
from spacy.training import Example

In [2]:
def read_menota_annotations_for_spacy():
    l = []
    with codecs.open(os.path.join("data-menota-spacy.json"), "r", encoding="utf-8") as f:
        for book in f:
            for line in json.loads(book):
                if all([i == "-" for i in line[0]]):
                    continue
                l.append(line)
    return l

In [3]:
data = read_menota_annotations_for_spacy()

In [4]:
print(f"Number of tokens {sum([len(i[0]) for i in data])}")

Number of tokens 173164


In [5]:
unique_tokens = set()
for i in data:
    unique_tokens.update(i[0])

print(f"Number of unique tokens {len(unique_tokens)}")

Number of unique tokens 16866


In [6]:
unique_tags = set()
for i in data:
    unique_tags.update(i[1]["pos"])
# print(unique_tags)
print(f"Number of unique tags {len(unique_tags)}")

Number of unique tags 1171


In [7]:
print("Example")
print(data[0][0])
print(data[0][1]["pos"])

Example
['í', 'vígskǫrðum', 'verja', ',', 'þá', 'er', 'gott', 'at', 'gera', 'hengivígskǫ', 'af', 'léttum', 'viði']
['xAP', 'xNC gN nP cD sI', 'xVB fI tPS vA', '-', 'xAV rP', 'xVB fF tPS mIN p3 nS vA', 'xAJ rP gN nS cN sI', 'xIM', 'xVB fI tPS vA', 'xNC gN nP cA sI', 'xAP', 'xAJ rP gM nS cD sI', 'xNC gM nS cD sI']


In [8]:
def create_vocab(training_data):
    vocabulary_set = set()
    for text, _ in training_data:
        for lemma in text:
            vocabulary_set.add(lemma)
    return vocabulary_set


def reduce_tags(pos_tag: str) -> str:
    first_part = pos_tag.strip().split(" ")[0]
    if "|" in first_part:
        return reduce_tags(first_part.split("|")[0])
    if first_part.startswith("x"):
        return first_part[1:]
    elif "00000" == first_part:
        return ""
    else:
        return pos_tag

    
def reduce_word(word: str):
    #return word.replace(" ", "").lower()
    if word == "-":
        return word
    elif word == ",–":
        return "-"
    elif word == ",–":
        return ","
    else:
        return word.lower().replace(" ", "").replace("-", "").replace("'", "")


def from_text_annotations_to_spacy_training_data(data):
    l = []
    for sentence, annotation in data:
        try:
            l.append(([reduce_word(token) for token in sentence],
                       dict(pos=[reduce_tags(tag) for tag in annotation["pos"]])))
        except AttributeError:
            print(sentence)
            print(annotation)
    return l

In [9]:
pos_training_data = from_text_annotations_to_spacy_training_data(data)

In [10]:
unique_tags = set()
for i in pos_training_data:
    unique_tags.update(i[1]["pos"])
# print(unique_tags)
print(f"Number of unique tags {len(unique_tags)}, {unique_tags}")

Number of unique tags 31, {'AV', 'VP', 'IT', 'AJ', 'NE', 'PD', 'AP', '-', 'RP', 'UA', 'DP', 'NO', 'NC', 'DQ', 'DD', 'EX', 'NA', 'CU', 'PR', 'PQ', 'VB', 'CC', 'PE', 'FN', 'AQ', 'AT', 'NP', 'IM', 'PI', 'FW', 'CS'}


In [11]:
print("Example")
print(f"{pos_training_data[0][0]}")
print(f"{pos_training_data[0][1]['pos']}")

Example
['í', 'vígskǫrðum', 'verja', ',', 'þá', 'er', 'gott', 'at', 'gera', 'hengivígskǫ', 'af', 'léttum', 'viði']
['AP', 'NC', 'VB', '-', 'AV', 'VB', 'AJ', 'IM', 'VB', 'NC', 'AP', 'AJ', 'NC']


In [32]:
def create_training(training_data):
    vocab = spacy.Vocab(string=create_vocab(training_data))
    nlp = spacy.blank("is")
    db = DocBin()  # create a DocBin object
    text = []
    k = 0
    for j, item in enumerate(training_data):
        words = item[0]
        pos = item[1]
#         example = Example.from_dict(Doc(vocab, words=words), pos)

        try:
            #print(j, len(words), len(pos["pos"]), repr(" ".join(words)))
            #print([i for i in zip(words, pos["pos"])])

            #example = Example.from_dict(nlp.make_doc(" ".join(words)), pos)
            example = Example.from_dict(nlp.make_doc(" ".join(words)), dict(words=words,tags=pos["pos"]))

            db.add(example.reference)
            text.append(" ".join(words))
        except ValueError:
            k += 1
            #print(len(words), " ".join(words))
            #print(len(pos["pos"]), pos)
            #print("problème")
            #break
    print(f"{k} problematic cases")
            
    return db, " ".join(text)

In [33]:
def save_training_data(training_data):
    random.shuffle(training_data)
    valid = training_data[:500]
    train = training_data[500:]

    train, train_corpus = create_training(train)
    train.to_disk("./data/train.spacy")
    with codecs.open("./data/train_corpus.txt", "w", encoding="utf-8") as f:
        f.write(train_corpus)

    valid, valid_corpus = create_training(valid)
    valid.to_disk("./data/valid.spacy")
    with codecs.open("./data/valid_corpus.txt", "w", encoding="utf-8") as f:
        f.write(valid_corpus)

In [34]:
save_training_data(pos_training_data)

9 problematic cases
2 problematic cases


In [35]:
pos_training_data[0]

(['hverir',
  'eftir',
  'skulu',
  'renna',
  ',',
  'er',
  'maðr',
  'er',
  'hǫggvinn',
  'á',
  'þingi',
  'capitulus',
  'en',
  'ef',
  'maðr',
  'verðr',
  'hǫggvinn',
  'á',
  '-',
  '-'],
 {'pos': ['DQ',
   'AP',
   'VB',
   'VB',
   '-',
   'CS',
   'NC',
   'VB',
   'VB',
   'AP',
   'NC',
   'FW',
   'CC',
   'CS',
   'NC',
   'VB',
   'VB',
   'AP',
   '-',
   '-']})

In [36]:
tags = []
for item in pos_training_data:
    labels = item[1]["pos"]
    for label in labels:
        tags.append(label)
tags = list(set(tags))
with open ("tag.json", "w", encoding='utf-8') as f:
    print(tags)
    json.dump(tags, f, indent=4)

['AV', 'VP', 'IT', 'NE', 'PD', 'AP', '-', 'RP', 'UA', 'DP', 'NO', 'NC', 'DQ', 'DD', 'EX', 'CS', 'CU', 'NA', 'PR', 'PQ', 'VB', 'CC', 'PE', 'FN', 'AT', 'AQ', 'NP', 'IM', 'PI', 'FW', 'AJ']


In [37]:
!python -m spacy debug data config-v3.2.cfg --paths.train ./data/train.spacy --paths.dev ./data/valid.spacy


============================ Data file validation ============================
corpora
[+] Pipeline can be initialized with data
[+] Corpus is loadable

=============================== Training stats ===============================
Language: is
Training pipeline: tok2vec, tagger
9577 training docs
498 evaluation docs
[!] 13 training examples also in evaluation data

============================== Vocab & Vectors ==============================
[i] 164000 total word(s) in the data (15372 unique)
[i] No word vectors present in the package

=========================== Part-of-speech Tagging ===========================
[i] 31 label(s) in train data

================================== Summary ==================================
[+] 2 checks passed
[!] 1 warning


In [38]:
!python -m spacy debug config config-v3.2.cfg --paths.train ./data/train.spacy --paths.dev ./data/valid.spacy


============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
corpora
[+] Config is valid


In [19]:
!python -m spacy train config-v3.2.cfg --paths.train ./data/train.spacy --paths.dev ./data/valid.spacy

^C
